# Partitioning a dataset into separate training and test datasets

Phần này giới thiệu ngắn gọn về concept chia dữ liệu thành các bộ dữ liệu phục vụ cho training và testing. Ghi nhớ rằng so sánh giá trị dự đoán được với nhãn true label trong tập test được hiểu là bước đánh giá **unbiased performance evaluation** cho model trước khi ta áp dụng nó vào trong thực tiễn. Trong phần này, ta sẽ chuẩn bị một dữ liệu mới, dữ liệu **Wine** dataset. Sau khi ta tiền xử lý chúng, ta có thể khám phá những kỹ thuật khác nhau cho **feature selection** để giảm số chiều của dữ liệu.

Dữ liệu về **Wine** là một bộ dữ liệu mở trong UCI machine learning repo ([https://archive.ics.uci.edu/ml/datasets/Wine](https://archive.ics.uci.edu/ml/datasets/Wine)); Dữ liệu bao gồm 178 mẫu rượu và 13 thuộc tính để mô tả các hóa chất khác nhau của mỗi loại.

### Obtaining the Wine dataset

Ta có thể tìm thấy bản copy của Wine dataset trên UCI server. Ví dụ, để load dữ liệu Wine dataset từ thư mục local, ta có thể thay thế dòng này:

df = pd.read_csv('https://archive.ics.uci.edu/ml/'
                 'machine-learning-databases/wine/wine.data',
                  header=None)
                  
Thành:
df = pd.read_csv('your/local/path/to/wine.data', header=None)



Sử dụng thư viện Pandas, ta trực tiếp đọc dữ liệu Wine từ UCI machine learning repository:



In [1]:
import numpy as np
import pandas as pd


df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                      'ml/machine-learning-databases/'
                      'wine/wine.data', header=None
                    )
df_wine.columns = ['Class label', 'Alcohol',
                    'Malic acid', 'Ash',
                    'Alcalinity of ash', 'Magnesium',
                    'Total phenols', 'Flavanoids',
                    'Nonflavanoid phenols',
                    'Proanthocyanins',
                    'Color intensity', 'Hue',
                    'OD280/OD315 of diluted wines',
                    'Proline']
print('Class labels ', np.unique(df_wine['Class label']))
print(df_wine.head())
df_wine.info()

Class labels  [1 2 3]
   Class label  Alcohol  Malic acid   Ash  Alcalinity of ash  Magnesium  \
0            1    14.23        1.71  2.43               15.6        127   
1            1    13.20        1.78  2.14               11.2        100   
2            1    13.16        2.36  2.67               18.6        101   
3            1    14.37        1.95  2.50               16.8        113   
4            1    13.24        2.59  2.87               21.0        118   

   Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins  \
0           2.80        3.06                  0.28             2.29   
1           2.65        2.76                  0.26             1.28   
2           2.80        3.24                  0.30             2.81   
3           3.85        3.49                  0.24             2.18   
4           2.80        2.69                  0.39             1.82   

   Color intensity   Hue  OD280/OD315 of diluted wines  Proline  
0             5.64  1.04          

13 thuộc tính khác nhau trong dữ liệu Wine, mô tả tính chất hóa học của 178 mẫu rượu, được liệt kê trong bảng ở trên.

Các mẫu thuộc về 3 lớp khác nhau 1, 2, 3 và liên quan đến ba loại nho được trồng tại cùng một vùng của Ý nhưng chuyển hóa thành các nhóm rựơụ khác nhau, được mô tả trong link: ([https://archive.
ics.uci.edu/ml/machine-learning-databases/wine/wine.names](https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.names)) 

Một cách tiện lợi để chia dữ liệu của chúng ta thành bộ test và training datasets đó là sử dụng train_test_split trong thư viện sklearn.model_selection module: 


In [2]:
from sklearn.model_selection import train_test_split
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y, test_size=0.3)

Ở đoạn code trên, đầu tiên ta gán dạng biểu diễn mảng NumPy cho các cột features columns 1-13 vào biến X và ta gán giá trị của nhãn từ cột thứ 1 của df_wine cho biến y. Sau đó, ta dùng hàm train_test_split để random cắt X và y thành 2 tập train và test data. Bằng cách thêm tham số test_size = 0.3, ta gán giá trị là 30% của mẫu rượu sẽ được gán cho biến X_test và y_test, và 70% mẫu còn lại sẽ được gán cho biến X_train và y_train. Thêm tham số stratify để đảm bảo rằng cả dữ liệu train hoặc test gì cũng có tỉ lệ phần trăm của các class là tương đương với dữ liệu chuẩn ban đầu.

### Chossing an appropriate ratio for partitioning a dataset into training and testing datasets

Nếu ta chia dữ liệu thành train và test dataset, ta ghi nhớ rằng ta đang nắm giữ những thông tin quý giá để cho thuật toán có thể học được. Do đó ta không nên cấp quá nhiều thông tin cho tập test set. Tuy nhiên, dữ liệu test quá nhỏ thì hàm ước lượng lỗi tổng quát càng tệ. Chia dữ liệu như thế nào thực chất là cân bằng sự đánh đổi, tỉ lệ chia phổ biến nhất là 60:40, 70:30 hoặc là 80:20, phụ thuộc vào kích thướng của tập ban đầu. Tuy nhiên đối với những dữ liệu lớn, tỉ lệ chia 90:10 hoặc là 99:1 là một cách tiếp cận thông thuộc. Ví dụ, nếu dataset có 100000 mẫu, thì ta nên giữ lại 10000 mẫu dùng cho test và để có thể có được giá trị ước lượng của hàm tổng quan có chất lượng tốt nhất. Thông tin thêm có thể được tìm thấy tại [https://arxiv.org/pdf/1811.12808.pdf](https://arxiv.org/pdf/1811.12808.pdf).

Hơn nữa, thay vì bỏ đi dữ liệu test mà ta dùng để kiểm tra, ta có thể sữ dụng lại chính chúng để cải thiện hiệu suất dự đoán cho model. Trong khi cách tiếp cận này là được khuyến khích, nhưng mà nó vẫn có thể dẫn đến hiệu suất tổng quát thấp khi mà dữ liệu dataset nhỏ và test dataset chứa ngoại lệ. Do đó, khi fitting model bằng toàn bộ dữ liệu, ta không còn tập dũ liệu độc lập nào để dánh gía hiệu suất của nó.
